# 9장 웹 애플리케이션에 머신 러닝 모델 내장

- 머신 러닝 모델의 현재 훈련 상태 저장하기
- SQLite 데이터베이스를 이용하여 데이터 저장하기
- 유명한 플라스크(Flask) 웹 프레임워크를 이용하여 웹 애플리케이션 개발하기
- 공개 웹 서버에 머신 러닝 애플리케이션 배포하기

## 9.0 영화 리뷰 분류를 위한 모델 훈련하기

In [1]:
import gzip

# 영화 리뷰 데이터 저장(Data 디렉토리에 csv file로 저장)
with gzip.open('Data/movie_data.csv.gz') as f_in, open('Data/movie_data.csv','wb') as f_out:
    f_out.writelines(f_in)
    
# 불용어 저장(~/nltk_data/corpora/stopwords 디렉토리에 저장)
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wooil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop = stopwords.words('english')
porter = PorterStemmer() # 

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

def stream_docs(path):
    with open(path, 'r', encoding='utf-8') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

In [3]:
next(stream_docs(path='Data/movie_data.csv'))

('"In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />""Murder in Greenwich"" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich f

In [4]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [5]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier

vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         tokenizer=tokenizer)

clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
doc_stream = stream_docs(path='Data/movie_data.csv')

In [6]:
import pyprind
pbar = pyprind.ProgBar(45)

classes = np.array([0, 1])
for _ in range(45):
    X_train, y_train = get_minibatch(doc_stream, size=1000)
    if not X_train:
        break
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:29


In [7]:
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_test = vect.transform(X_test)
print('정확도: %.3f' % clf.score(X_test, y_test))

정확도: 0.868


In [8]:
clf = clf.partial_fit(X_test, y_test)
clf

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='log', max_iter=1,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=1, shuffle=True, tol=0.001, validation_fraction=0.1,
              verbose=0, warm_start=False)

## 9.1 학습된 사이킷런 추정기 저장

- [pickle 모듈 공식 문서](https://docs.python.org/3.7/library/pickle.html)  
**파이썬 객체**의 구조를 압축된 **바이트코드(bytecode)**로 직렬화하고 복원할 수 있음  


- **(참고사항)**  
직렬화(serialization) : 데이터나 객체의 현재 상태를 디스크에 저장하거나 네트워크로 전송할 수 있도록 변환하는 과정.  
joblib 모듈 : numpy array 객체를 직렬화할 때 효율적인 모듈 [공식문서](https://joblib.readthedocs.io/en/latest/)

In [9]:
import pickle
import os

# 피클 파일 저장할 디렉토리 생성
dest = os.path.join('movieclassifier', 'pkl_objects')
if not os.path.exists(dest):
    os.makedirs(dest)

# 불용어 저장
pickle.dump(stop, open(os.path.join(dest, 'stopwords.pkl'), 'wb'), protocol=4)

# 모델 저장
pickle.dump(clf, open(os.path.join(dest, 'classifier.pkl'), 'wb'), protocol=4)

### movieclassifier/vectorizer.py 파일 생성 (추후 임포트하여 사용하기 위함)

```python
from sklearn.feature_extraction.text import HashingVectorizer
import re
import os
import pickle

cur_dir = os.path.dirname(__file__)
stop = pickle.load(open(
                os.path.join(cur_dir, 
                'pkl_objects', 
                'stopwords.pkl'), 'rb'))

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) \
                   + ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

vect = HashingVectorizer(decode_error='ignore',
                         n_features=2**21,
                         preprocessor=None,
                         tokenizer=tokenizer)
```

### 테스트를 위해 커널 재시작

이 단계에서 커널을 초기화한 후 아래 코드를 실행하면 앞서 저장했던 모델 객체가 불러와집니다.

In [10]:
import os
os.chdir('movieclassifier')
import pickle
import re
from vectorizer import vect # 저장한 불용어 객체는 여기서 불러와집니다.

clf = pickle.load(open(os.path.join('pkl_objects', 'classifier.pkl'), 'rb'))

In [11]:
import numpy as np
label = {0:'부정적', 1:'긍정적'}

example = [
           'I love this movie',           # 1 (긍정적)
           'This movie is too boring.',   # 0 (부정적)
           'Not bad.',                    # 0 (부정적)
           'Awesome!!!',                  # 1 (긍정적)
           ]

X = vect.transform(example)

pred = clf.predict(X)
proba = clf.predict_proba(X)*100

for i in range(len(pred)):
    print("리뷰: {} \n예측: {} \n확률: {}% \n".format(example[i], label[pred[i]], np.round(np.max(proba[i]),1)))

리뷰: I love this movie 
예측: 긍정적 
확률: 81.4% 

리뷰: This movie is too boring. 
예측: 부정적 
확률: 97.0% 

리뷰: Not bad. 
예측: 부정적 
확률: 99.6% 

리뷰: Awesome!!! 
예측: 긍정적 
확률: 83.2% 



## 9.2 데이터를 저장하기 위해 SQLite 데이터베이스 설정

In [12]:
os.getcwd()

'D:\\github\\GDG_ML_Study\\ch09\\movieclassifier'

In [13]:
import sqlite3

conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()
c.execute('DROP TABLE IF EXISTS review_db')
c.execute('CREATE TABLE review_db (review TEXT, sentiment INTEGER, date TEXT)')

example1 = 'I love this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example1, 1))

example2 = 'I disliked this movie'
c.execute("INSERT INTO review_db (review, sentiment, date) VALUES (?, ?, DATETIME('now'))", (example2, 0))

conn.commit()
conn.close()

In [14]:
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()

c.execute("SELECT * FROM review_db WHERE date BETWEEN '2017-01-01 10:10:10' AND DATETIME('now')")
results = c.fetchall()

conn.close()

In [15]:
print(results)

[('I love this movie', 1, '2020-02-03 08:20:11'), ('I disliked this movie', 0, '2020-02-03 08:20:11')]


## 9.3 플라스크 웹 애플리케이션 개발

[Flask 공식 문서](http://flask.pocoo.org/docs)

### 9.3.1 첫 번째 플라스크 애플리케이션

#### 디렉토리 생성
```
1st_flask_app_1/
    app.py
    templates/
        first_app.html
```

#### app.py

```python
from flask import Flask, render_template

app = Flask(__name__)
@app.route('/')
def index():
    return render_template('first_app.html')

if __name__=='__main__':
    app.run()
```

#### first_app.html

```html
<!doctype html>
<html>
  <head>
    <title>첫 번째 애플리케이션</title>
  </head>
  <body>

  <div>
	와우, 첫 번째 플라스크 웹 애플리케이션입니다!
  </div>

  </body>
</html>
```

#### 로컬에서 플라스크 애플리케이션 실행하기

- `app.py`파일이 위치한 디렉토리에서 터미널을 열고, 다음 명령어를 실행

```bash
python app.py
```

- Output

```bash
 * Serving Flask app "app" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
```

- Output의 마지막 줄에 출력된 `http://127.0.0.1:5000`을 웹 브라우저에서 실행

### 9.3.2 폼 검증과 화면 출력

In [16]:
# WTFoarms 라이브러리 설치
# !pip install wtforms

#### 디렉토리 생성
```
1st_flask_app_2/
    app.py
    static/
        style.css
    templates/
        _formhelpers.html
        first_app.html
        hello.html
```

#### app.py

```python
from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators

app = Flask(__name__)

class HelloForm(Form):
    sayhello = TextAreaField('',[validators.DataRequired()])

@app.route('/')
def index():
    form = HelloForm(request.form)
    return render_template('first_app.html', form=form)

@app.route('/hello', methods=['POST'])
def hello():
    form = HelloForm(request.form)
    if request.method == 'POST' and form.validate():
        name = request.form['sayhello']
        return render_template('hello.html', name=name)
    return render_template('first_app.html', form=form)

if __name__ == '__main__':
    app.run(debug=True)
```

#### _formhelpers.html

```html
{% macro render_field(field) %}
  <dt>{{ field.label }}
  <dd>{{ field(**kwargs)|safe }}
  {% if field.errors %}
    <ul class=errors>
    {% for error in field.errors %}
      <li>{{ error }}</li>
    {% endfor %}
    </ul>
  {% endif %}
  </dd>
  </dt>
{% endmacro %}
```

#### style.css

```css
body {
	font-size: 2em;
}
```

#### first_app.html

```html
<!doctype html>
<html>
  <head>
    <title>첫 번째 애플리케이션</title>
	<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  </head>
  <body>

{% from "_formhelpers.html" import render_field %}

<div>이름을 입력해 주세요.</div>
<form method=post action="/hello">

  <dl>
	  {{ render_field(form.sayhello) }}
  </dl>

  <input type=submit value='이름 입력' name='submit_btn'>

</form>

  </body>
</html>
```


#### hello.html

```html
<!doctype html>
<html>
  <head>
    <title>첫 번째 애플리케이션</title>
	<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
  </head>
  <body>

<div>{{ name }} 님 안녕하세요!</div>


  </body>
</html>
```

## 9.4 영화 리뷰 분류기를 웹 애플리케이션으로 만들기

#### 디렉토리 생성

[Directory](https://github.com/machinelearning-pangyo/python-machine-learning-book-2nd-edition/tree/master/Ch09_%EC%9B%B9%20%EC%95%A0%ED%94%8C%EB%A6%AC%EC%BC%80%EC%9D%B4%EC%85%98%EC%97%90%20%EB%A8%B8%EC%8B%A0%20%EB%9F%AC%EB%8B%9D%20%EB%AA%A8%EB%8D%B8%20%EB%82%B4%EC%9E%A5/movieclassifier)

```
movieclassifier/
    pkl_objects/
        classifier.pkl
        stopwords.pkl
    static/
        style.css
    templates/
        _formhelpers.html
        results.html
        reviewform.html
        thanks.html
        
    app.py
    reviews.sqlite
    vectorizer.py
```

### 웹으로 리뷰 작성 후 DB 확인

In [17]:
conn = sqlite3.connect('reviews.sqlite')
c = conn.cursor()

c.execute("SELECT * FROM review_db WHERE date BETWEEN '2017-01-01 10:10:10' AND DATETIME('now')")
results = c.fetchall()

conn.close()
results

[('I love this movie', 1, '2020-02-03 08:20:11'),
 ('I disliked this movie', 0, '2020-02-03 08:20:11')]

## 9.5 공개 서버에 웹 애플리케이션 배포

### 9.5.1 PythonAnywhere 계정 만들기

[Pythonanywhere](https://www.pythonanywhere.com/)

### 9.5.2 영화 분류 애플리케이션 업로드

#### Pythonanywhere - 'Web' 탭

- Add a new web app 클릭
- Next 클릭
- Flask 선택
- Python 3.7 선택
- /home/`USER NAME`/app.py 입력 (app.py 파일이 기본값으로 초기화되므로, 파일을 업로드 해줘야합니다.)


#### Pythonanywhere - 'File' 탭

- 수동으로 웹 애플리케이션이 있는 로컬 디렉토리 내 파일들을 업로드(가장 확실)
- 교재 내용 참고하여 github repo를 bash로 복사하면 오류 발생했음.

#### Pythonanywhere - 'Web' 탭

- Reload 버튼 클릭


[공개 서버에서 배포 중인 웹 애플리케이션](http://wooil.pythonanywhere.com/)

### 9.5.3 영화 분류기 업데이트

#### 디렉토리 생성

[Directory](https://github.com/machinelearning-pangyo/python-machine-learning-book-2nd-edition/tree/master/Ch09_%EC%9B%B9%20%EC%95%A0%ED%94%8C%EB%A6%AC%EC%BC%80%EC%9D%B4%EC%85%98%EC%97%90%20%EB%A8%B8%EC%8B%A0%20%EB%9F%AC%EB%8B%9D%20%EB%AA%A8%EB%8D%B8%20%EB%82%B4%EC%9E%A5/movieclassifier_with_update)


```
movieclassifier_with_update/
    pkl_objects/
        classifier.pkl
        stopwords.pkl
    static/
        style.css
    templates/
        _formhelpers.html
        results.html
        reviewform.html
        thanks.html
        
    app.py
    reviews.sqlite
    update.py
    vectorizer.py
```